# Creating plots of in-situ and simulation data
***
In this example, plotting data from the simulation together with spacecraft in-situ observations is demonstrated.

## Plotting virtual spacecraft data  

The easiest way to compare spacecraft observations and simulation data is if the position of the spacecraft or observation point has been tracked by the simulation. In this case, the simulation has produced an ASCII `.dsv` file that contains data from the virtual spacecraft and that can be directly used. 

Typically, the simulation includes Earth as a virtual spacecraft, so a file ending with `_Earth.dsv` should be available. 

To showchase this, let's explore the data file contained in this directory. The file can easily be read in as a pandas DataFrame object:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DayLocator, HourLocator, DateFormatter
%matplotlib inline

In [ ]:
df = pd.read_csv("june_2015_Earth.dsv", sep='\s+', parse_dates=['date'])

In [ ]:
# Compute output cadence in minutes for plotting purposes
output_cadence = (df['date']-df['date'].shift()).median().seconds/60.0

There are several possibilities for plotting the observed data. For non-realtime data, a very good option is to use the [ai.cdas](https://bitbucket.org/isavnin/ai.cdas) library:

In [ ]:
from ai import cdas

Let's use OMNI data. Note that the data includes bad values, which need to be removed by setting them to NaN.

In [ ]:
omni_5min = cdas.get_data('istp_public', 
                               'OMNI_HRO_5MIN',
                               df['date'].iloc[0],
                               df['date'].iloc[-1],
                              ['proton_density', 'Vx', 'Vy', 'Vz', 'BX_GSE', 'BY_GSE', 'BZ_GSE', 'flow_speed'])

Let's plot the speed:

In [ ]:
fig, ax = plt.subplots(figsize=(14.0,3.0))

ax.plot(omni_5min['EPOCH_TIME'], [v if v != (99999.9) else np.NaN for v in omni_5min['FLOW_SPEED,_GSE']], '-r', label='OMNI 5 min')
ax.plot(df['date'], df['vr[km/s]'], 'b', label='EUHFORIA {:g} min'.format(output_cadence))
ax.set_ylabel("Speed [km/s]")
ax.xaxis.set_minor_locator(DayLocator())
ax.legend(loc='upper left');
#plt.savefig("vr_ar_Earth.png")

and the number density:

In [ ]:
fig, ax = plt.subplots(figsize=(14.0,3.0))

ax.plot(omni_5min['EPOCH_TIME'], [n if n != (999.99) else np.NaN for n in omni_5min['PROTON_DENSITY']], '-r', label='OMNI 5 min')
ax.plot(df['date'], df['n[1/cm^3]'], 'b', label='EUHFORIA {:g} min'.format(output_cadence))
ax.set_ylabel("Number density [cm$^{-3}$]")
ax.xaxis.set_minor_locator(DayLocator())
ax.legend(loc='upper left');
#plt.savefig("number_density_at_Earth.png")

## Real time data

While the CDA system is great, it takes some time (often several months) for spacecraft data to be available. For this reason, a faster route is often needed. 

Near real-time data for ACE is accessible as follows: 

In [ ]:
import datetime

In [ ]:
import euhforia.core.rt

In [ ]:
ace_nrt = euhforia.core.rt.ACERealTimeData('ace_data/')

In [ ]:
ace_nrt.start_time = datetime.datetime.now() - datetime.timedelta(days=10)
ace_nrt.end_time   = datetime.datetime.now()

In [ ]:
ace_nrt.retrieve()

Speed:

In [ ]:
fig, ax = plt.subplots(figsize=(14, 3))
ax.plot(ace_nrt.date, ace_nrt.bulk_speed/1e3, '-r', label="ACE NRT", lw=2, alpha=0.7)
ax.set_ylabel("Speed [km/s]")
ax.legend(loc='upper left');

Number density:

In [ ]:
fig, ax = plt.subplots(figsize=(14, 3))
ax.plot(ace_nrt.date, ace_nrt.proton_number_density/1e6, '-m', label="ACE NRT", lw=2, alpha=0.7)
ax.set_ylabel("Number density [cm$^{-3}$]")
ax.legend(loc='upper left');

Magnetic field components

In [ ]:
fig, ax = plt.subplots(figsize=(14, 3))
ax.plot(ace_nrt.date, ace_nrt.Bx/1e-9, '-', color='#1f77b4', label="Bx ACE NRT", lw=2, alpha=0.7)
ax.plot(ace_nrt.date, ace_nrt.By/1e-9, '-', color='#ff7f0e', label="By ACE NRT", lw=2, alpha=0.7)
ax.plot(ace_nrt.date, ace_nrt.Bz/1e-9, '-', color='#2ca02c', label="Bz ACE NRT", lw=2, alpha=0.7)

ax.set_ylabel("Magnetic field [nT]")
ax.legend(loc='upper left');

Temperature

In [ ]:
fig, ax = plt.subplots(figsize=(14, 3))
ax.plot(ace_nrt.date, ace_nrt.ion_temperature/1e6, '-', color='#ffbb78', label="ACE NRT", lw=2, alpha=0.7)

ax.set_ylabel("Ion temperature [MK]")
ax.legend(loc='upper left');

***